1. O que acontecerá na execução dos algoritmos apresentados para a solução do problema da Mochila 0/1 se o número de itens crescer de forma demasiada? Por que isso ocorre? Tente aumentar gradualmente o número de itens e verifique o que ocorre. Relate.


A complexidade é $O(2^n)$, então o tempo de execução cresce exponencialmente com o número de itens, pois a fase de `expand` gera 2 estados (0 e 1) para cada item atual, e é chamada recursivamente para cada um desses estados. Isso faz com que o tempo de execução cresça exponencialmente com o número de itens, tornando o algoritmo impraticável para um número grande de itens.

In [9]:
type State = list[int]

def step(state: State) -> list[State]:
    """Perform one step of searching, splitting states into all their possible states.

    Complexity is $O(1)$ as we are just iterating over a fixed number of states."""
    POSSIBLE_STATES = range(2)
    return [state + [i] for i in POSSIBLE_STATES]

def expand(states: list[State]) -> list[State]:
    """Expand into all possible next (1 distance) states.

    Complexity is $O(n)$ where $n$ is len(states)."""
    if not states:
        return step([])
    return [s for state in states for s in step(state)]

def combine(size: int) -> list[State]:
    """Combine all possible states of a given size.

    Complexity is $O(2^len(size))$ as we are generating all possible states."""
    states = []
    for _ in range(size):
        states = expand(states)
    return states

def size(state: State, weights: list[int]) -> int:
    """Calculate the size of a state.

    Complexity is $O(n)$ where $n$ is len(state)."""
    return sum([state[i] * weights[i] for i in range(len(state))])

def valid(state: State, weights: list[int], limit: int) -> bool:
    """Check if a state is valid.

    Complexity is same as [`size`]."""
    return size(state, weights) <= limit

def solution(state: State, weights: list[int]) -> list[int]:
    """Returns all sizes that are in the solution.

    Complexity is $O(n)$ where $n$ is `len(state)`."""
    return [state[i] * weights[i] for i in range(len(state)) if state[i] > 0]

def blindSearchKnapsack01(
    knapsack_size: int, weights: list[int]
) -> list[tuple[State, int]]:
    """Blind search for the knapsack problem.

    Complexity is $O(2^n)$ where $n$ is `len(weights)`."""
    return [
        (solution(state, weights), size(state, weights))
        for state in combine(len(weights))
        if valid(state, weights, knapsack_size)
    ]

%reload_ext ipython_unittest

In [10]:
%%unittest

assert blindSearchKnapsack01(3, [1, 7, 8]) == [([], 0), ([1], 1)]
assert blindSearchKnapsack01(8, [1, 7, 8]) == [([], 0), ([8], 8), ([7], 7), ([1], 1), ([1, 7], 8)]

Success

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [11]:
import time, random

random_item_size = lambda: random.randint(1, 15)

start = time.time()
blindSearchKnapsack01(8, [random_item_size() for _ in range(3)])
time_taken = time.time() - start
print(f"Execution time for size 3:\t{time_taken * 1000:.2f}ms")

start = time.time()
blindSearchKnapsack01(25, [random_item_size() for _ in range(10)])
time_taken = time.time() - start
print(f"Execution time for size 10:\t{time_taken * 1000:.2f}ms")

start = time.time()
blindSearchKnapsack01(37, [random_item_size() for _ in range(15)])
time_taken = time.time() - start
print(f"Execution time for size 15:\t{time_taken * 1000:.2f}ms")

start = time.time()
blindSearchKnapsack01(37, [random_item_size() for _ in range(20)])
time_taken = time.time() - start
print(f"Execution time for size 20:\t{time_taken * 1000:.2f}ms")

Execution time for size 3:	0.06ms
Execution time for size 10:	1.06ms
Execution time for size 15:	71.04ms
Execution time for size 20:	2313.03ms


2. Considere que no problema da mochila 0/1 cada um dos itens disponíveis tem um valor e que o objetivo é encher a mochila para maximizar o valor agregado dos ítens colocados na mochila. Refaça a solução em python apresentada anteriormente para resolver este problema.


In [12]:
def blindSearchKnapsack01Max(
    knapsack_size: int, weights: list[int]
) -> list[tuple[State, int]]:
    """Blind search for the knapsack problem, returning the maximum size.
    
    Complexity is $O(2^n)$ where $n$ is `len(weights)`."""
    all_solutions = blindSearchKnapsack01(knapsack_size, weights)
    max_sieve = max(all_solutions, key=lambda x: x[1])[1]
    filter_func = lambda x: x[1] == max_sieve
    return list(filter(filter_func, all_solutions))

In [13]:
%%unittest

assert blindSearchKnapsack01Max(8, [1, 7, 4]) == [([1, 7], 8)]
assert blindSearchKnapsack01Max(8, [1, 7, 8]) == [([8], 8), ([1, 7], 8)]

Success

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

3. Implemente o algoritmo de agrupamento hierárquico divisivo para dados unidimensionais.


In [32]:
import numpy as np

type Cluster = np.ndarray

def hierarchical_clustering(
    elements: Cluster, k: int
) -> list[Cluster]:
    """Splits the elements hierarchically into k clusters."""
    clusters = [sorted(elements)]
    while len(clusters) < k:
        # print(f"Clusters: {clusters}")
        # Pick the one with highest variance to split
        cluster = max(clusters, key=lambda x: np.var(x))
        # Split it into two
        clusters.remove(cluster)
        clusters.extend(np.array_split(cluster, 2))
    return clusters

In [45]:
%%unittest

data = np.array([1, 9, 2, 4, 5, 5, 8, 8, 9, 1, 5])
clusters = sorted(hierarchical_clustering(data, 3), key=lambda x: x[0])
cluster_lists = [list(cluster) for cluster in clusters]
assert cluster_lists == [[1,1,2], [4,5,5], [5,8,8,9,9]]

Success

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

4. Altere o algoritmo de agrupamento hierárquico divisivo para considerar dados multidimensionais numéricos. Indique o que necessita ser feito para aplicar o algoritmo da questão 10 aqui. Qual a maior dificuldade para isso? Proponha uma solução para contornar essa dificuldade. Indique casos em que essa solução pode
não ser satisfatória. Implemente a sua solução.

Precisamos considerar números em conjuntos, ao invés de números isolados. Ou seja, toda comparação deve ser feita com conjuntos de números ao invés de números separados.

A maior dificuldade seria de definir uma função de distância adequada, por exemplo a distância Euclidiana. Uma solução para contornar essa dificuldade seria considerar somente o maior valor de cada conjunto, ou seja, a distância entre dois conjuntos seria a distância entre o maior valor de cada conjunto. Isso pode não ser satisfatório quando os conjuntos são muito diferentes, pois a distância entre os maiores valores pode não ser representativa da distância entre os elementos do mesmo conjunto, mas isso é dependente do domínio do problema.

Exemplo: $\{\{1, 2, 3\}, \{4, 5, 6\}, \{7, 8, 9\}\}$ seria considerado como $\{3, 6, 9\}$.

In [46]:
type Cluster = np.ndarray[np.ndarray]

# TODO

def hierarchical_clustering_multidimensional(
    elements: Cluster, k: int
) -> list[Cluster]:
    """Splits the elements hierarchically into k clusters."""
    clusters = [sorted(elements)]
    while len(clusters) < k:
        # print(f"Clusters: {clusters}")
        # Pick the one with highest variance to split
        cluster = max(clusters, key=lambda x: np.var(x))
        # Split it into two
        clusters.remove(cluster)
        clusters.extend(np.array_split(cluster, 2))
    return clusters

In [48]:
%%unittest

data = np.array([1, 9, 2, 4, 5, 5, 8, 8, 9, 1, 5])
clusters = sorted(hierarchical_clustering_multidimensional(data, 3), key=lambda x: x[0])
cluster_lists = [list(cluster) for cluster in clusters]
assert cluster_lists == [[1,1,2], [4,5,5], [5,8,8,9,9]]

data = np.array([[1, 3], 9, 2, 4, 5, 5, 8, 8, 9, 1, 5])
clusters = sorted(hierarchical_clustering_multidimensional(data, 3), key=lambda x: x[0])
cluster_lists = [list(cluster) for cluster in clusters]
assert cluster_lists == [[1,1,2], [4,5,5], [5,8,8,9,9]]

Success

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>